# Import libraries

In [1]:
from google.cloud import bigquery
import os
from google.oauth2 import service_account
import pandas as pd
import pickle
import seaborn as sns

# Obtain data

Set up client connection to bigquery api

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= None
client = bigquery.Client()

Get bigquery 'World Bank WDI' dataset

In [3]:
dataset_id = client.dataset("world_bank_wdi", project="bigquery-public-data")
dataset = client.get_dataset(dataset_id)

View tables in dataset

In [4]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

country_series_definitions
country_summary
footnotes
indicators_data
series_summary
series_time


View indicators_data table

In [5]:
table_id = dataset_id.table("indicators_data")
table = client.get_table(table_id)
[print(item) for item in table.schema]
client.list_rows(table, max_results=5).to_dataframe()

SchemaField('country_name', 'STRING', 'NULLABLE', '', (), None)
SchemaField('country_code', 'STRING', 'NULLABLE', '', (), None)
SchemaField('indicator_name', 'STRING', 'NULLABLE', '', (), None)
SchemaField('indicator_code', 'STRING', 'NULLABLE', '', (), None)
SchemaField('value', 'FLOAT', 'NULLABLE', '', (), None)
SchemaField('year', 'INTEGER', 'NULLABLE', '', (), None)


/Users/blantj/opt/anaconda3/lib/python3.7/site-packages/google/cloud/bigquery/table.py:1690: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,country_name,country_code,indicator_name,indicator_code,value,year
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,76.258066,2002
1,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,79.851681,2006
2,Arab World,ARB,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,83.035000,2007
3,Arab World,ARB,"Adjusted net enrollment rate, primary, male (%...",SE.PRM.TENR.MA,87.285710,2010
4,Arab World,ARB,Adjusted net national income (annual % growth),NY.ADJ.NNTY.KD.ZG,4.627110,2017


Query anomoly detection dataset from indicators_data table

In [7]:
indicators_query = """
        SELECT *
        FROM `bigquery-public-data.world_bank_wdi.indicators_data`
        WHERE indicator_name IN ('Adjusted net national income per capita (current US$)', 
                                 'Inflation, consumer prices (annual %)',
                                 'Labor force, female (% of total labor force)',
                                 'Urban population (% of total population)',
                                 'Population growth (annual %)',
                                 'Employment in services (% of total employment) (modeled ILO estimate)',
                                 'Total debt service (% of GNI)',
                                 'Compulsory education, duration (years)',
                                 'Gross savings (% of GNI)',
                                 'Hospital beds (per 1,000 people)')"""
indicators_query_job = client.query(indicators_query)
indicators_df = indicators_query_job.to_dataframe()
indicators_df.to_csv('Excel/dataset.csv', index=False)

# Scrub data

Read df with indicators data into pandas

In [15]:
df = pd.read_csv('Excel/dataset.csv')
df.head()

,country_name,country_code,indicator_name,indicator_code,value,year
0,East Asia & Pacific (excluding high income),EAP,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,35.555550,1996
1,East Asia & Pacific (excluding high income),EAP,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,34.845872,2000
2,Eritrea,ERI,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,17.164867,1999
3,Guinea-Bissau,GNB,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,5.103593,2009
4,Hungary,HUN,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,20.204166,1995


Remove rows with regional and other non-country country_names (which are specified in exclusions.csv) from the dataset

In [16]:
exclusions_df = pd.read_csv('Excel/exclusions.csv')
df.set_index('country_code', inplace=True)
df.drop(exclusions_df['country_code'].values, inplace=True)
df.reset_index(inplace=True)
df.head()

,country_code,country_name,indicator_name,indicator_code,value,year
0,ERI,Eritrea,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,17.164867,1999
1,GNB,Guinea-Bissau,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,5.103593,2009
2,HUN,Hungary,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,20.204166,1995
3,NZL,New Zealand,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,19.410751,2007
4,GMB,"Gambia, The",Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,10.734246,1983


View number of repeated indicator measurements for the same country across different years

In [18]:
df['country_indicator'] = df['country_name'] + ' ' + df['indicator_name']
df['country_indicator'].value_counts()

Kenya Urban population (% of total population)          60
South Sudan Urban population (% of total population)    60
Mongolia Urban population (% of total population)       60
Korea, Rep. Urban population (% of total population)    60
Jamaica Urban population (% of total population)        60
                                                        ..
South Sudan Gross savings (% of GNI)                     2
Eritrea Hospital beds (per 1,000 people)                 2
Namibia Hospital beds (per 1,000 people)                 2
South Africa Hospital beds (per 1,000 people)            1
Tuvalu Hospital beds (per 1,000 people)                  1
Name: country_indicator, Length: 1953, dtype: int64

Create list specifying which df rows have the most recent annual measurements for their country/indicator combination

In [19]:
def to_keep(row):
    values = list(df[df['country_indicator']==row['country_indicator']]['year'].values)
    if row['year'] == max(values):
        return True
    else:
        return False
    
row_booleans = df.apply(to_keep, axis=1)
df['keep'] = row_booleans

Filter out df rows with outdated country indicator measurements

In [20]:
pre_pivot_df = df[df['keep']==True]
pre_pivot_df.drop(columns=['keep'], inplace=True)
print(pre_pivot_df.shape)
pre_pivot_df.head()

(1953, 7)


/Users/blantj/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,country_code,country_name,indicator_name,indicator_code,value,year,country_indicator
35,HUN,Hungary,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,28.992088,2019,Hungary Gross savings (% of GNI)
65,CYP,Cyprus,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,14.531399,2019,Cyprus Gross savings (% of GNI)
72,ETH,Ethiopia,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,33.312217,2018,Ethiopia Gross savings (% of GNI)
89,GBR,United Kingdom,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,13.790267,2019,United Kingdom Gross savings (% of GNI)
110,ITA,Italy,Gross savings (% of GNI),NY.GNS.ICTR.GN.ZS,20.763017,2019,Italy Gross savings (% of GNI)


Pivot indicator measurements table to countries as row indices, and indicators as column indices

In [21]:
wb_df = pre_pivot_df.pivot(index='country_name', columns='indicator_name', values='value')
wb_df.head()

indicator_name,Adjusted net national income per capita (current US$),"Air transport, passengers carried","Compulsory education, duration (years)",Employment in services (% of total employment) (modeled ILO estimate),Gross savings (% of GNI),"Hospital beds (per 1,000 people)","Inflation, consumer prices (annual %)","Labor force, female (% of total labor force)",Population growth (annual %),Total debt service (% of GNI),Urban population (% of total population)
country_name,,,,,,,,,,,
Afghanistan,458.370738,1722612.61,9.0,39.396000,18.558666,0.5,2.302373,21.618689,2.311817,0.286377,25.754
Albania,4201.077084,303137.00,9.0,43.662998,14.438802,2.9,1.411091,41.471734,-0.426007,4.299708,61.229
Algeria,3098.851733,6442442.00,10.0,59.638000,38.383538,1.9,1.951768,17.671559,1.933950,0.117442,73.189
Andorra,NaN,NaN,10.0,NaN,NaN,2.5,NaN,NaN,0.176454,NaN,87.984
Angola,1028.564719,1516628.00,6.0,41.730999,27.280625,0.8,17.145320,50.293109,3.242983,11.020077,66.177


Perform wb_df.info() to understand what further scrubbing needed

In [22]:
wb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 196 entries, Afghanistan to Zimbabwe
Data columns (total 11 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Adjusted net national income per capita (current US$)                  180 non-null    float64
 1   Air transport, passengers carried                                      181 non-null    float64
 2   Compulsory education, duration (years)                                 183 non-null    float64
 3   Employment in services (% of total employment) (modeled ILO estimate)  178 non-null    float64
 4   Gross savings (% of GNI)                                               171 non-null    float64
 5   Hospital beds (per 1,000 people)                                       191 non-null    float64
 6   Inflation, consumer prices (annual %)                                  181 non-n

Drop 'Total debt service (% of GNI)' column due to the large number of missing values

In [87]:
wb_df.drop(columns=['Total debt service (% of GNI)', 'Air transport, passengers carried'], inplace=True)
wb_df.head()

indicator_name,Adjusted net national income per capita (current US$),"Compulsory education, duration (years)",Employment in services (% of total employment) (modeled ILO estimate),Gross savings (% of GNI),"Hospital beds (per 1,000 people)","Inflation, consumer prices (annual %)","Labor force, female (% of total labor force)",Population growth (annual %),Urban population (% of total population)
country_name,,,,,,,,,
Afghanistan,458.370738,9.0,39.396000,18.558666,0.5,2.302373,21.618689,2.311817,25.754
Albania,4201.077084,9.0,43.662998,14.438802,2.9,1.411091,41.471734,-0.426007,61.229
Algeria,3098.851733,10.0,59.638000,38.383538,1.9,1.951768,17.671559,1.933950,73.189
Andorra,NaN,10.0,NaN,NaN,2.5,NaN,NaN,0.176454,87.984
Angola,1028.564719,6.0,41.730999,27.280625,0.8,17.145320,50.293109,3.242983,66.177


Fill 'Compulsory education, duration (years)' column missing values with column mean

In [88]:
wb_df['Compulsory education, duration (years)'] = wb_df['Compulsory education, duration (years)'].fillna(value=
                            wb_df['Compulsory education, duration (years)'].dropna().values.mean())

Drop rows with more than 1 missing value

In [89]:
wb_df = wb_df[wb_df.isnull().apply(lambda x:x.sum(), axis=1)<2]

Replace missing 'Adjusted net national income per capita (current US$)' value with value from alternative souce

In [90]:
wb_df.loc['Malta','Adjusted net national income per capita (current US$)'] = 27290

Replace missing 'Hospital beds (per 1,000 people)' value with value from alternative souce

In [91]:
wb_df.loc['South Sudan', 'Hospital beds (per 1,000 people)'] = .8

Replace missing 'Inflation, consumer prices (annual %)' value with the feature mean

In [92]:
wb_df['Inflation, consumer prices (annual %)'] = wb_df['Inflation, consumer prices (annual %)'].fillna(value = 
                            wb_df['Inflation, consumer prices (annual %)'].mean())

Replace missing 'Gross savings (% of GNI)' with feature mean

In [98]:
wb_df['Gross savings (% of GNI)'] = wb_df['Gross savings (% of GNI)'].fillna(value = 
                            wb_df['Gross savings (% of GNI)'].mean())

Confirm that data is scrubbed with wb_df.info()

In [99]:
wb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174 entries, Afghanistan to Zimbabwe
Data columns (total 9 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Adjusted net national income per capita (current US$)                  174 non-null    float64
 1   Compulsory education, duration (years)                                 174 non-null    float64
 2   Employment in services (% of total employment) (modeled ILO estimate)  174 non-null    float64
 3   Gross savings (% of GNI)                                               174 non-null    float64
 4   Hospital beds (per 1,000 people)                                       174 non-null    float64
 5   Inflation, consumer prices (annual %)                                  174 non-null    float64
 6   Labor force, female (% of total labor force)                           174 non-nu

save scrubbed wb_df to csv file for use in modeling file

In [102]:
wb_df.reset_index().to_csv('Scrubbed_wb_dataset.csv', index=False)